In [1]:
#include <stdlib.h>
#include <math.h>
#include <string.h>
#include <time.h>
#include <cuda.h>

#define MAX_ITER 1000
#define X_RESN  1920
#define Y_RESN  1080

__global__ void mandelbrot(int *d_image, int max_iter, int x_resn, int y_resn, float x_offset, float y_offset, float zoom)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    int k = i + j * x_resn;
    float x0 = (i - x_resn/2) / (0.5 * zoom * x_resn) + x_offset;
    float y0 = (j - y_resn/2) / (0.5 * zoom * y_resn) + y_offset;
    float x = 0.0;
    float y = 0.0;
    int iter = 0;
    while (x*x + y*y < 4 && iter < max_iter) {
        float xtemp = x*x - y*y + x0;
        y = 2*x*y + y0;
        x = xtemp;
        iter++;
    }
    d_image[k] = iter;
}

int main(int argc, char *argv[]) {
    int *h_image, *d_image;
    int max_iter = MAX_ITER;
    int x_resn = X_RESN;
    int y_resn = Y_RESN;
    float x_offset = 0.0;
    float y_offset = 0.0;
    float zoom = 1.0;
    int i, j, k;
    int size = x_resn * y_resn * sizeof(int);
    FILE *fp;
    char *filename = "mandelbrot.pam";
    char *comment = "# ";

    // allocate host memory
    h_image = (int *) malloc(size);

    // allocate device memory
    cudaMalloc((void **) &d_image, size);

    // setup execution parameters
    dim3
    threads(16, 16);
    dim3
    grid(x_resn / threads.x, y_resn / threads.y);

    // create and start timer
    clock_t start, stop;
    start = clock();

    // execute the kernel
    mandelbrot<<< grid, threads >>>(d_image, max_iter, x_resn, y_resn, x_offset, y_offset, zoom);

    // stop and destroy timer
    stop = clock();
    double timer_seconds = ((double) (stop - start)) / CLOCKS_PER_SEC;
    printf("Time to generate:  %f s \r ", timer_seconds);
    
    // copy result from device to host
    cudaMemcpy(h_image, d_image, size, cudaMemcpyDeviceToHost);
    
    // write result to file
    fp = fopen(filename, "w");
    fprintf(fp, "P7 %s %d %d %d %d ", comment, x_resn, y_resn, 255, 0);
    for (j = 0; j < y_resn; j++) {
        for (i = 0; i < x_resn; i++) {
            k = i + j * x_resn;
            fprintf(fp, "%c%c%c", h_image[k] % 256, h_image[k] % 256, h_image[k] % 256);
        }
    }
    fclose(fp);
    
    // cleanup memory
    free(h_image);
    cudaFree(d_image);
    
    return 0;
}

SyntaxError: ignored